In [32]:
from datetime import datetime, timedelta

def get_date_range_by_offset(offset):
    today = datetime.today().replace(day=1)  # Start from the 1st of the current month
    start_date = (today - timedelta(days=30 * offset)).replace(day=1)
    next_month = (start_date + timedelta(days=31)).replace(day=1)
    end_date = (next_month - timedelta(days=1))
    return start_date.strftime('%d-%m-%Y'), end_date.strftime('%d-%m-%Y')

get_date_range_by_offset(1)


('01-11-2024', '30-11-2024')

In [ ]:
import pandas as pd
from connections import get_mongo_client
from mssql import max_number_find, insert_into_mssql

client = get_mongo_client()

# Select the database and collection
db = client['task']
task_collection = db['task']

# Define the query
query = {
    'data.katm_333.return.data.general_cbr.loans.loan': {'$exists': True}
    # , 'number': {'$gt': max_num} 
}

# Define the projection
projection = {
    'number': 1, 
    'request.clientId': 1,
    'data.katm_333.return.data.general_cbr.loans.loan': 1

}

# Fetch the documents that match the query
docs = task_collection.find(query, projection).sort('number', -1).limit(1)
document = list(docs)[0]

In [32]:
document

{'_id': ObjectId('67386b2b87c0a8e9d7a359e7'),
 'type': 'credit_mikro',
 'request': {'newClient': 0,
  'retailOrgId': '0189f448-7274-1228-b5f9-aa88f2364a51',
  'amount': 3000000000.0,
  'clientId': '7250990',
  'rate': 24,
  'requestId': '01933462-6a54-18fc-9097-f560c8b89eb2',
  'smsPhone': '998906990699',
  'credProgId': '60024_6',
  'timeout_sec': 120,
  'statement': 'prescoring',
  'term': 60},
 'payload': {'newClient': 0,
  'retailOrgId': '0189f448-7274-1228-b5f9-aa88f2364a51',
  'amount': 30000000,
  'clientId': '7250990',
  'rate': 24,
  'requestId': '01933462-6a54-18fc-9097-f560c8b89eb2',
  'smsPhone': '998906990699',
  'credProgId': '60024_6',
  'timeout_sec': 120,
  'statement': 'prescoring',
  'term': 60},
 'expire': datetime.datetime(2024, 11, 16, 9, 53, 39, 596000),
 'status': 'error',
 'keywords': ['7250990', '998906990699', '30302892180059', 'AC0166423'],
 'created': datetime.datetime(2024, 11, 16, 9, 51, 39, 596000),
 'modified': datetime.datetime(2024, 11, 16, 9, 51, 39,

### collaterals table with specified dtypes

In [1]:
import json
import pyodbc
import pandas as pd
from pymongo import MongoClient

In [18]:
# Function to insert DataFrame into MSSQL in chunks
def insert_into_mssql(df, table_name):
    driver = 'ODBC Driver 17 for SQL Server'
    server = '172.17.17.22,54312'
    database = 'RISKDB'
    username = 'SMaksudov'
    password = 'CfhljhVfrc#'
    
    conn = pyodbc.connect(
        f"Driver={{{driver}}};"
        f"Server={server};"
        f"Database={database};"
        f"UID={username};"
        f"PWD={password};"
    )

    cursor = conn.cursor()

    for index, row in df.iterrows():
        try:
            insert_query = f"INSERT INTO {table_name} VALUES ({', '.join(['?' for _ in range(len(df.columns))])})"
            # Ensure the 'value' field is formatted as a DECIMAL
            values = [row[col] if col != 'value' else float(row[col]) if row[col] else None for col in df.columns]
            cursor.execute(insert_query, tuple(values))
        except Exception as e:
            print(f"Error inserting row {index}: {e}")

    conn.commit()
    cursor.close()
    conn.close()

In [8]:
# Function to get the maximum 'number' value from the table
def max_number_find():
    driver = 'ODBC Driver 17 for SQL Server'
    server = '172.17.17.22,54312'
    database = 'RISKDB'
    username = 'SMaksudov'
    password = 'CfhljhVfrc#'

    conn = pyodbc.connect(
        f"Driver={{{driver}}};"
        f"Server={server};"
        f"Database={database};"
        f"UID={username};"
        f"PWD={password};"
    )

    cursor = conn.cursor()

    check_table_query = """select max(number) from RISKDB.dbo.katm_333_collaterals_dtypes;"""
    cursor.execute(check_table_query)
    result = cursor.fetchone()

    max_number = int(result[0] if result else None)
    return max_number


In [11]:
max_num = max_number_find()

In [26]:
# MongoDB client setup
client = MongoClient(
    'mongodb://172.17.39.13:27017',
    username='Sardor.Maksudov',
    password='jDS3pqTV',
    authSource='admin'
)

# Select the database and collection
db = client['task']
task_collection = db['task']

# Define the query and projection
query = {
    'data.katm_333.return.data.general_cbr.loans.loan': {'$exists': True},
    'number': {'$gt': 7000000}
}

# projection = {
#     'number': 1, 
#     'request.clientId': 1,
#     'data.katm_333.return.data.general_cbr.loans.loan': 1
# }


In [27]:
# Fetch documents from MongoDB
docs = task_collection.find(query, projection).limit(1)

In [28]:
document = list(docs)[0]

In [19]:
# Loop through MongoDB documents
for doc in docs:
    data = []  # Initialize data list for each document
    
    if doc:  # Ensure the document is not None
        number = doc.get('number')
        client_id = doc.get('request', {}).get('clientId', None)
        loans = doc.get('data', {}).get('katm_333', {}).get('return', {}).get('data', {}).get('general_cbr', {}).get('loans', {}).get('loan', [])

        # Normalize loans to a list
        if isinstance(loans, dict):
            loans = [loans]
        
        for loan in loans:
            uuid = loan.get('uuid')
            collaterals = loan.get('collaterals', None)

            # Skip empty collaterals
            if not collaterals or (isinstance(collaterals, (list, dict)) and not collaterals):
                continue

            # Process collaterals when it's a dictionary
            if isinstance(collaterals, dict):
                collateral = collaterals.get('collateral')
                
                if isinstance(collateral, dict):  # Single collateral case
                    row = {
                        'number': number,
                        'uuid': uuid,
                        'client_id': int(client_id) if client_id else None,
                        'collateral_code': int(collateral.get('collateral_code', 0)) if collateral.get('collateral_code') else None,
                        'collateral_id': str(collateral.get('collateral_id', '')),
                        'open_date': pd.to_datetime(collateral.get('open_date'), errors='coerce').date() if collateral.get('open_date') else None,
                        'value': float(collateral.get('value', 0)) if collateral.get('value') else None,
                        'currency': str(collateral.get('currency', '')),
                        'end_date': pd.to_datetime(collateral.get('end_date'), errors='coerce').date() if collateral.get('end_date') else None,
                        'fact_end_date': pd.to_datetime(collateral.get('fact_end_date'), errors='coerce').date() if collateral.get('fact_end_date') else None
                    }
                    data.append(row)

                elif isinstance(collateral, list):  # List of collaterals case
                    for item in collateral:
                        row = {
                            'number': number,
                            'uuid': uuid,
                            'client_id': int(client_id) if client_id else None,
                            'collateral_code': int(item.get('collateral_code', 0)) if item.get('collateral_code') else None,
                            'collateral_id': str(item.get('collateral_id', '')),
                            'open_date': pd.to_datetime(item.get('open_date'), errors='coerce').date() if item.get('open_date') else None,
                            'value': float(item.get('value', 0)) if item.get('value') else None,
                            'currency': str(item.get('currency', '')),
                            'end_date': pd.to_datetime(item.get('end_date'), errors='coerce').date() if item.get('end_date') else None,
                            'fact_end_date': pd.to_datetime(item.get('fact_end_date'), errors='coerce').date() if item.get('fact_end_date') else None
                        }
                        data.append(row)

        # If valid data rows exist, insert them into SQL Server
        if data:
            df = pd.DataFrame(data)
            insert_into_mssql(df, 'katm_333_collaterals_dtypes')


In [14]:
row

NameError: name 'row' is not defined

In [ ]:
import pandas as pd
import pyodbc
from pymongo import MongoClient

# Function to insert DataFrame into SQL Server
def insert_into_mssql(df, table_name):
    driver = 'ODBC Driver 17 for SQL Server'
    server = '172.17.17.22,54312'
    database = 'RISKDB'
    username = 'SMaksudov'
    password = 'CfhljhVfrc#'

    conn = pyodbc.connect(
        f"Driver={{{driver}}};"
        f"Server={server};"
        f"Database={database};"
        f"UID={username};"
        f"PWD={password};"
    )
    cursor = conn.cursor()

    # Ensure schema-qualified table name
    schema, table = table_name.split('.')
    full_table_name = f"[{schema}].[{table}]"

    # Create table if it doesn't exist
    cursor.execute(f"""
    IF OBJECT_ID(N'{full_table_name}', 'U') IS NULL
    CREATE TABLE {full_table_name} (
        number INT NULL,
        uuid NVARCHAR(100),
        client_id INT NULL,
        collateral_code INT NULL,
        collateral_id NVARCHAR(100),
        open_date DATE NULL,
        value FLOAT NULL,
        currency NVARCHAR(50),
        end_date DATE NULL,
        fact_end_date DATE NULL
    );
    """)
    conn.commit()

    # Insert data
    placeholders = ', '.join(['?' for _ in df.columns])
    insert_query = f"INSERT INTO {full_table_name} VALUES ({placeholders})"
    for _, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))
    conn.commit()
    cursor.close()
    conn.close()

# MongoDB connection
client = MongoClient(
    'mongodb://172.17.39.13:27017',
    username='Sardor.Maksudov',
    password='jDS3pqTV',
    authSource='admin'
)
db = client['task']
task_collection = db['task']

# Find the maximum 'number' from SQL Server
def get_max_number():
    driver = 'ODBC Driver 17 for SQL Server'
    server = '172.17.17.22,54312'
    database = 'RISKDB'
    username = 'SMaksudov'
    password = 'CfhljhVfrc#'

    conn = pyodbc.connect(
        f"Driver={{{driver}}};"
        f"Server={server};"
        f"Database={database};"
        f"UID={username};"
        f"PWD={password};"
    )
    cursor = conn.cursor()
    cursor.execute("SELECT MAX(number) FROM dbo.katm_333_collaterals_dtypes")
    result = cursor.fetchone()
    return int(result[0]) if result and result[0] else 0

# Flatten nested MongoDB documents
def process_collaterals(doc):
    data = []
    number = doc.get('number')
    client_id = doc.get('request', {}).get('clientId')
    loans = doc.get('data', {}).get('katm_333', {}).get('return', {}).get('data', {}).get('general_cbr', {}).get('loans', {}).get('loan', [])

    # Ensure loans is a list
    if isinstance(loans, dict):
        loans = [loans]

    stack = [(loan, []) for loan in loans]  # Start with loans

    while stack:
        current, path = stack.pop()
        uuid = current.get('uuid')
        collaterals = current.get('collaterals')

        if not collaterals:
            continue

        # Handle collaterals dictionary or list
        if isinstance(collaterals, dict):
            collaterals = [collaterals]
        for collateral in collaterals:
            nested_collateral = collateral.get('collateral')
            if isinstance(nested_collateral, list):
                stack.extend([(item, path + [uuid]) for item in nested_collateral])
            elif isinstance(nested_collateral, dict):
                row = {
                    'number': number,
                    'uuid': uuid,
                    'client_id': client_id,
                    'collateral_code': nested_collateral.get('collateral_code'),
                    'collateral_id': nested_collateral.get('collateral_id'),
                    'open_date': nested_collateral.get('open_date'),
                    'value': nested_collateral.get('value'),
                    'currency': nested_collateral.get('currency'),
                    'end_date': nested_collateral.get('end_date'),
                    'fact_end_date': nested_collateral.get('fact_end_date')
                }
                data.append(row)
    return data

# Main Process
max_number = get_max_number()
query = {'data.katm_333.return.data.general_cbr.loans.loan': {'$exists': True}, 'number': {'$gt': 7000000}}
projection = {
    'number': 1,
    'request.clientId': 1,
    'data.katm_333.return.data.general_cbr.loans.loan': 1
}

all_data = []

# Fetch and process MongoDB documents
for doc in task_collection.find(query, projection):
    all_data.extend(process_collaterals(doc))

if all_data:
    df = pd.DataFrame(all_data)
    insert_into_mssql(df, 'dbo.katm_333_collaterals_dtypes')


ProgrammingError: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'dbo.katm_333_collaterals_dtypes'. (208) (SQLExecDirectW)")

In [52]:
import json
import pyodbc
import pandas as pd
from bson import ObjectId
from datetime import datetime
from pymongo import MongoClient

# Function to insert DataFrame into MSSQL in chunks
def insert_into_mssql(df, table_name):
    driver = 'ODBC Driver 17 for SQL Server'
    server = '172.17.17.22,54312'
    database = 'RISKDB'
    username = 'SMaksudov'
    password = 'CfhljhVfrc#'
    
    conn = pyodbc.connect(
        f"Driver={{{driver}}};"
        f"Server={server};"
        f"Database={database};"
        f"UID={username};"
        f"PWD={password};"
    )

    cursor = conn.cursor()

    # Check if table exists, if not create it
    check_table_query = f"""
    IF OBJECT_ID(N'{table_name}', 'U') IS NULL
    BEGIN
        CREATE TABLE {table_name}  (
            number INT,
            uuid NVARCHAR(50),
            client_id INT,
            collateral_code INT,
            collateral_id NVARCHAR(50),
            open_date DATE,
            value FLOAT,
            currency NVARCHAR(50),
            end_date DATE,
            fact_end_date DATE
        );
    END;"""
    cursor.execute(check_table_query)
    conn.commit()

    # Insert the data from DataFrame into the table
    for index, row in df.iterrows():
        try:
            # Prepare date fields by converting them into the correct format
            open_date = row['open_date']
            end_date = row['end_date']
            fact_end_date = row['fact_end_date']

            def convert_date(date_str):
                if date_str:
                    try:
                        # First, check if the date is in the YYYYMMDD format (without dots)
                        if len(date_str) == 8 and date_str.isdigit():
                            # Convert YYYYMMDD to DD.MM.YYYY
                            date_str = f"{date_str[6:8]}.{date_str[4:6]}.{date_str[:4]}"

                        # Check if the date is in DD.MM.YY format (2-digit year)
                        if len(date_str) == 8 and date_str[2] == '.' and date_str[5] == '.':
                            # Convert DD.MM.YY to DD.MM.YYYY
                            day, month, year = date_str.split('.')
                            year = '20' + year  # Assume the year is 20xx (for 18.07.22, treat as 2018)
                            date_str = f"{day}.{month}.{year}"

                        # Now try parsing the date with the DD.MM.YYYY format
                        date_obj = datetime.strptime(date_str, '%d.%m.%Y')
                        return date_obj.strftime('%Y-%m-%d')  # Return in YYYY-MM-DD format for SQL Server
                    except Exception as e:
                        print(f"Error converting date: {date_str} - {e}")
                return None  # Return None if the date is invalid or empty

            open_date = convert_date(open_date)
            end_date = convert_date(end_date)
            fact_end_date = convert_date(fact_end_date)

            # Insert data into SQL Server
            insert_query = f"""
            INSERT INTO {table_name} (number, uuid, client_id, collateral_code, collateral_id, open_date, value, currency, end_date, fact_end_date)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            values = (
                row['number'],
                row['uuid'],
                row['client_id'],
                row['collateral_code'],
                row['collateral_id'],
                open_date,
                row['value'],
                row['currency'],
                end_date,
                fact_end_date
            )
            cursor.execute(insert_query, values)
        except Exception as e:
            print(f"Error inserting row {index}: {e}")

    conn.commit()
    cursor.close()
    conn.close()

def max_number_find():
    driver = 'ODBC Driver 17 for SQL Server'
    server = '172.17.17.22,54312'
    database = 'RISKDB'
    username = 'SMaksudov'
    password = 'CfhljhVfrc#'

    conn = pyodbc.connect(
        f"Driver={{{driver}}};"
        f"Server={server};"
        f"Database={database};"
        f"UID={username};"
        f"PWD={password};"
    )

    cursor = conn.cursor()

    check_table_query = f"""SELECT MAX(number) FROM katm_333_collaterals"""
    cursor.execute(check_table_query)
    result = cursor.fetchone()
    max_number = int(result[0] if result else 0)
    return max_number

max_num = max_number_find()

client = MongoClient(
    'mongodb://172.17.39.13:27017',
    username='Sardor.Maksudov',
    password='jDS3pqTV',
    authSource='admin'
)

# Select the database and collection
db = client['task']
task_collection = db['task']

# Define the query
query = {
    'data.katm_333.return.data.general_cbr.loans.loan': {'$exists': True},
    # 'number': {'$gt': max_num}
}

# Define the projection
projection = {
    'number': 1, 
    'request.clientId': 1,
    'data.katm_333.return.data.general_cbr.loans.loan': 1
}

# Fetch the documents
docs = task_collection.find(query, projection)

# Loop through MongoDB documents and prepare data for SQL Server
for doc in docs:
    data = []
    if doc:
        number = doc.get('number')
        client_id = doc.get('request', {}).get('clientId', {})
        loans = doc.get('data', {}).get('katm_333', {}).get('return', {}).get('data', {}).get('general_cbr', {}).get('loans', {}).get('loan', [])

        if isinstance(loans, dict):
            loans = [loans]
        
        for loan in loans:
            uuid = loan.get('uuid')
            collaterals = loan.get('collaterals')

            if not collaterals or collaterals == '' or (isinstance(collaterals, (list, dict)) and not collaterals):
                continue

            if isinstance(collaterals, dict):
                collateral = collaterals.get('collateral')
                if isinstance(collateral, dict):
                    row = {
                        'number': number,
                        'uuid': uuid,
                        'client_id': client_id,
                        'collateral_code': collateral.get('collateral_code', ''),
                        'collateral_id': collateral.get('collateral_id', ''),
                        'open_date': collateral.get('open_date', ''),
                        'value': collateral.get('value', ''),
                        'currency': collateral.get('currency', ''),
                        'end_date': collateral.get('end_date', ''),
                        'fact_end_date': collateral.get('fact_end_date', '')
                    }
                    data.append(row)

                elif isinstance(collateral, list):
                    for item in collateral:
                        row = {
                            'number': number,
                            'uuid': uuid,
                            'client_id': client_id,
                            'collateral_code': item.get('collateral_code', ''),
                            'collateral_id': item.get('collateral_id', ''),
                            'open_date': item.get('open_date', ''),
                            'value': item.get('value', ''),
                            'currency': item.get('currency', ''),
                            'end_date': item.get('end_date', ''),
                            'fact_end_date': item.get('fact_end_date', '')
                        }
                        data.append(row)

        if data:
            df = pd.DataFrame(data)
            insert_into_mssql(df, 'gold.katm_333_collaterals')


# NEXT TEST

In [1]:
import pandas as pd
from utils import convert_date
from connections import get_mongo_client, get_sql_server_connection

In [2]:
client = client = get_mongo_client()
db = client['task']
task_collection = db['task']
query = {
    'data.katm_333.return.data.general_cbr.loans.loan': {'$exists': True}
    # , 'number': {'$gt': max_num} 
}
projection = {
    'number': 1, 
    'request.clientId': 1,
    'data.katm_333.return.data.general_cbr.loans.loan': 1

}

In [3]:
docs = task_collection.find_one(query, projection)

In [4]:
rows = []

number = docs.get('number')
client_id = docs.get('request', {}).get('clientId', {})
loans = docs.get('data', {}).get('katm_333', {}).get('return', {}).get('data', {}).get('general_cbr', {}).get('loans', {}).get('loan', [])       

if isinstance(loans, dict):
    loans = [loans]

for loan in loans:
    row = {'client_id': client_id, 'number': number, **loan}
    rows.append(row)
    
df = pd.DataFrame(rows)

In [5]:
df

,client_id,number,ttl_delq_90_119,ttl_delq_8_29,other_past_due,collaterals,ttl_delq_60_89,type,uuid,initial_pmt,...,principal_past_due,relationship,fact_close_date,is_own,outstanding,next_pmt_principal_date,coborrowers_count,principal_repaid_amount,guarantors,inf_confirm_date
0,6854981,7090120,,,0,,,32,033-00223-1753248,104300,...,0,1,20.10.2022,1,0,,0,2000000,,20.10.2022
1,6854981,7090120,,,0,,,32,033-00223-1815967,66921,...,0,1,27.07.2022,1,0,,0,1300000,,27.07.2022
2,6854981,7090120,,,0,,,32,033-00223-2066974,54839,...,0,1,13.04.2023,1,0,,0,1400000,,13.04.2023
3,6854981,7090120,,,,,,32,033-00223-2077710,46703,...,0,1,17.07.2023,1,0,,0,1200000,,17.07.2023
4,6854981,7090120,,,,,,32,033-00223-2258249,116003,...,0,1,,1,1882653.07,04.11.2024,0,1117346.93,,10.10.2024
5,6854981,7090120,,,,,,32,033-00223-1976292,52541,...,0,1,17.07.2023,1,0,,0,1000000,,17.07.2023
6,6854981,7090120,,,,,,32,033-00223-2534886,81793,...,0,1,,1,1581116.97,08.11.2024,0,418883.03,,10.10.2024
7,6854981,7090120,,,,,,32,033-00223-2628865,97831,...,0,1,,1,2001112.46,01.11.2024,0,298887.54,,10.10.2024
8,6854981,7090120,,,,,,32,033-00223-2754352,148298,...,,1,12.08.2024,1,0,,0,3400000,,12.08.2024
9,6854981,7090120,,,,,,32,033-00223-2815488,89549,...,,1,,1,2100000,04.11.2024,0,0,,10.10.2024


In [7]:
for i in df.columns:
    print(i)

client_id
number
ttl_delq_90_119
ttl_delq_8_29
other_past_due
collaterals
ttl_delq_60_89
type
uuid
initial_pmt
collateral_code
monthly_details
ttl_delq_120_plus
interest_rate
principal_repayment_amount
pmt_string_60m
next_pmt_principal
final_pmt_date
open_date
next_pmt
max_delq_balance
previous_pmt
business_category
ensured_amount
last_payment_date
ttl_delq_7
cur_to_base_limit
curr_balance_amt
status
ttl_delq_30_59
next_pmt_interest_date
delq_balance
pmt_string_start
pmt_freq
payments
other_outstanding
current_delq
int_past_due
principal_outstanding
next_payment_date
credit_limit
next_pmt_interest
currency
int_outstanding
principal_past_due
relationship
fact_close_date
is_own
outstanding
next_pmt_principal_date
coborrowers_count
principal_repaid_amount
guarantors
inf_confirm_date


In [8]:
def load_loan_columns(file_path):
    with open(file_path, 'r') as file:
        columns = file.read().splitlines()
    return columns

In [9]:
def map_dff_to_loan_columns(dff, loan_columns):
    final_df = pd.DataFrame(columns=loan_columns)
    
    # Fill in the columns of `final_df` with values from `dff`
    for col in final_df.columns:
        if col in dff.columns:
            final_df[col] = dff[col]  # Copy the values from `dff`
        else:
            # final_df[col] = None  # Set the column to None if not in `dff`
            final_df.drop(columns=[col], inplace=True) 
    
    return final_df

loans_columns_file = 'katm_333_general_cbr_loans_fields.txt'
loan_columns = load_loan_columns(loans_columns_file)
final_df = map_dff_to_loan_columns(df, loan_columns)

In [11]:
final_df.columns

Index(['number', 'client_id', 'uuid', 'inf_confirm_date', 'currency',
       'relationship', 'open_date', 'final_pmt_date', 'fact_close_date',
       'type', 'credit_limit', 'cur_to_base_limit', 'outstanding',
       'pmt_string_60m', 'status', 'next_pmt', 'previous_pmt', 'initial_pmt',
       'ttl_delq_7', 'ttl_delq_8_29', 'ttl_delq_30_59', 'ttl_delq_60_89',
       'ttl_delq_90_119', 'ttl_delq_120_plus', 'pmt_freq', 'delq_balance',
       'max_delq_balance', 'is_own', 'current_delq', 'pmt_string_start',
       'interest_rate', 'last_payment_date', 'next_payment_date',
       'curr_balance_amt', 'collateral_code', 'business_category',
       'principal_outstanding', 'principal_past_due', 'int_outstanding',
       'int_past_due', 'other_outstanding', 'other_past_due', 'ensured_amount',
       'coborrowers_count', 'next_pmt_principal', 'next_pmt_principal_date',
       'next_pmt_interest', 'next_pmt_interest_date',
       'principal_repayment_amount', 'principal_repaid_amount'],
      dt

In [12]:
final_df

,number,client_id,uuid,inf_confirm_date,currency,relationship,open_date,final_pmt_date,fact_close_date,type,...,other_outstanding,other_past_due,ensured_amount,coborrowers_count,next_pmt_principal,next_pmt_principal_date,next_pmt_interest,next_pmt_interest_date,principal_repayment_amount,principal_repaid_amount
0,7090120,6854981,033-00223-1753248,20.10.2022,UZS,1,11.10.2021,06.10.2023,20.10.2022,32,...,,0,2600000,0,,,,,0,2000000
1,7090120,6854981,033-00223-1815967,27.07.2022,UZS,1,27.12.2021,22.12.2023,27.07.2022,32,...,,0,1690000,0,,,,,0,1300000
2,7090120,6854981,033-00223-2066974,13.04.2023,UZS,1,04.11.2022,03.11.2025,13.04.2023,32,...,,0,1820000,0,,,13467.75,04.11.2024,0,1400000
3,7090120,6854981,033-00223-2077710,17.07.2023,UZS,1,14.11.2022,11.11.2025,17.07.2023,32,...,,,1560000,0,,,11016.19,04.11.2024,0,1200000
4,7090120,6854981,033-00223-2258249,10.10.2024,UZS,1,20.06.2023,19.06.2026,,32,...,,,3900000,0,78085.48,04.11.2024,39613.08,04.11.2024,1882653.07,1117346.93
5,7090120,6854981,033-00223-1976292,17.07.2023,UZS,1,26.08.2022,23.08.2024,17.07.2023,32,...,,,1300000,0,,,,,0,1000000
6,7090120,6854981,033-00223-2534886,10.10.2024,UZS,1,21.12.2023,18.12.2026,,32,...,,,2600000,0,47554.49,08.11.2024,35161.88,08.11.2024,1581116.97,418883.03
7,7090120,6854981,033-00223-2628865,10.10.2024,UZS,1,01.03.2024,26.02.2027,,32,...,,,2990000,0,48300.03,01.11.2024,49326.05,01.11.2024,2001112.46,298887.54
8,7090120,6854981,033-00223-2754352,12.08.2024,UZS,1,12.07.2024,09.07.2027,12.08.2024,32,...,,,4420000,0,,,92190.32,14.10.2024,0,3400000
9,7090120,6854981,033-00223-2815488,10.10.2024,UZS,1,23.09.2024,22.09.2027,,32,...,,,,0,34396.4,04.11.2024,57056.14,04.11.2024,2100000,0


In [14]:
print(final_df.columns)

Index(['number', 'client_id', 'uuid', 'inf_confirm_date', 'currency',
       'relationship', 'open_date', 'final_pmt_date', 'fact_close_date',
       'type', 'credit_limit', 'cur_to_base_limit', 'outstanding',
       'pmt_string_60m', 'status', 'next_pmt', 'previous_pmt', 'initial_pmt',
       'ttl_delq_7', 'ttl_delq_8_29', 'ttl_delq_30_59', 'ttl_delq_60_89',
       'ttl_delq_90_119', 'ttl_delq_120_plus', 'pmt_freq', 'delq_balance',
       'max_delq_balance', 'is_own', 'current_delq', 'pmt_string_start',
       'interest_rate', 'last_payment_date', 'next_payment_date',
       'curr_balance_amt', 'collateral_code', 'business_category',
       'principal_outstanding', 'principal_past_due', 'int_outstanding',
       'int_past_due', 'other_outstanding', 'other_past_due', 'ensured_amount',
       'coborrowers_count', 'next_pmt_principal', 'next_pmt_principal_date',
       'next_pmt_interest', 'next_pmt_interest_date',
       'principal_repayment_amount', 'principal_repaid_amount'],
      dt

In [15]:
table_name = 'gold.katm_333_general_cbr_loans'

In [38]:
def insert_into_mssql(df, table_name=table_name):
    conn = get_sql_server_connection()
    cursor = conn.cursor()

    # Insert the data from DataFrame into the table
    for index, row in df.iterrows():
        try:
        # Insert data into SQL Server
            insert_query = f"""
                INSERT INTO {table_name} VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
            ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
            ?, ?, ?, ?, ?, ?)

            """
            values = (
            row['number'],                           # int
            row['client_id'],                        # int
            row.get('uuid', 'missing-uuid'),         # nvarchar
            convert_date(row['inf_confirm_date']),  # date
            row['currency'],                         # nvarchar
            row['relationship'],                     # int
            convert_date(row['open_date']),          # date
            convert_date(row['final_pmt_date']),     # date
            convert_date(row['fact_close_date']),    # date
            row['type'],                             # int
            row['credit_limit'],                     # float
            row['cur_to_base_limit'],                # float
            row['outstanding'],                      # float
            row['pmt_string_60m'],                   # nvarchar
            row['status'],                           # int
            row['next_pmt'],                         # float
            row['previous_pmt'],                     # float
            row['initial_pmt'],                      # float
            row['ttl_delq_7'],                       # int
            row['ttl_delq_8_29'],                    # int
            row['ttl_delq_30_59'],                   # int
            row['ttl_delq_60_89'],                   # int
            row['ttl_delq_90_119'],                  # int
            row['ttl_delq_120_plus'],                # int
            row['pmt_freq'],                         # int
            row['delq_balance'],                     # float
            row['max_delq_balance'],                 # float
            row['is_own'],                           # int
            row['current_delq'],                     # int
            convert_date(row['pmt_string_start']),   # date
            row['interest_rate'],                    # float
            convert_date(row['last_payment_date']),  # date
            convert_date(row['next_payment_date']),  # date
            row['curr_balance_amt'],                 # float
            row['collateral_code'],                  # int
            row['business_category'],                # int
            row['principal_outstanding'],            # float
            row['principal_past_due'],               # float
            row['int_outstanding'],                  # float
            row['int_past_due'],                     # float
            row['other_outstanding'],                # float
            row['other_past_due'],                   # float
            row['ensured_amount'],                   # float
            row['coborrowers_count'],                # int
            row['next_pmt_principal'],               # float
            row['next_pmt_principal_date'],          # float
            row['next_pmt_interest'],                # float
            convert_date(row['next_pmt_interest_date']), # date
            row['principal_repayment_amount'],       # float
            row['principal_repaid_amount']           # float
            )
            cursor.execute(insert_query, values)
        except Exception as e:
            print(f"Error inserting row {index}: {e}")

    conn.commit()
    cursor.close()
    conn.close()

In [37]:
insert_into_mssql(final_df)

# NEXT TEST

In [2]:
import pandas as pd
from connections import get_mongo_client, get_sql_server_connection
from utils import convert_date, load_columns, map_dff_to_columns, max_number_find


In [3]:
loans_columns_file = 'katm_333_collaterals.txt'
table_name = 'bronze.katm_333_collaterals'

In [4]:
client = client = get_mongo_client()

In [5]:
db = client['task']
task_collection = db['task']
query = {
    'data.katm_333.return.data.general_cbr.loans.loan': {'$exists': True},
    # 'number': {'$gt': max_num}
}
projection = {
    'number': 1, 
    'request.clientId': 1,
    'data.katm_333.return.data.general_cbr.loans.loan': 1
}

In [6]:
doc = task_collection.find_one(query, projection)

In [10]:
doc

{'_id': ObjectId('670a32af48748061e051a874'),
 'request': {'clientId': '6854981'},
 'number': 7090120,
 'data': {'katm_333': {'return': {'data': {'general_cbr': {'loans': {'loan': [{'ttl_delq_90_119': '',
         'ttl_delq_8_29': '',
         'other_past_due': '0',
         'collaterals': '',
         'ttl_delq_60_89': '',
         'type': '32',
         'uuid': '033-00223-1753248',
         'initial_pmt': '104300',
         'collateral_code': '51',
         'monthly_details': {'monthly_detail': [{'date': '31.10.2021',
            'next_pmt': '104746.65',
            'pmt_pat': '0',
            'max_delq_balance': '',
            'worst_status': '',
            'outstanding': '2000000',
            'delq_balance': '',
            'history_date': '29.10.2021',
            'status': '',
            'balance_amt': ''},
           {'date': '30.11.2021',
            'next_pmt': '104746.65',
            'pmt_pat': '00',
            'max_delq_balance': '71085.19',
            'worst_status':

In [7]:
rows = []

number = doc.get('number')
client_id = doc.get('request', {}).get('clientId', {})
loans = doc.get('data', {}).get('katm_333', {}).get('return', {}).get('data', {}).get('general_cbr', {}).get('loans', {}).get('loan', [])       

if isinstance(loans, dict):
    loans = [loans]

for loan in loans:
    row = {'client_id': client_id, 'number': number, **loan}
    rows.append(row)

In [8]:
df = pd.DataFrame(rows)

In [9]:
df

,client_id,number,ttl_delq_90_119,ttl_delq_8_29,other_past_due,collaterals,ttl_delq_60_89,type,uuid,initial_pmt,...,principal_past_due,relationship,fact_close_date,is_own,outstanding,next_pmt_principal_date,coborrowers_count,principal_repaid_amount,guarantors,inf_confirm_date
0,6854981,7090120,,,0,,,32,033-00223-1753248,104300,...,0,1,20.10.2022,1,0,,0,2000000,,20.10.2022
1,6854981,7090120,,,0,,,32,033-00223-1815967,66921,...,0,1,27.07.2022,1,0,,0,1300000,,27.07.2022
2,6854981,7090120,,,0,,,32,033-00223-2066974,54839,...,0,1,13.04.2023,1,0,,0,1400000,,13.04.2023
3,6854981,7090120,,,,,,32,033-00223-2077710,46703,...,0,1,17.07.2023,1,0,,0,1200000,,17.07.2023
4,6854981,7090120,,,,,,32,033-00223-2258249,116003,...,0,1,,1,1882653.07,04.11.2024,0,1117346.93,,10.10.2024
5,6854981,7090120,,,,,,32,033-00223-1976292,52541,...,0,1,17.07.2023,1,0,,0,1000000,,17.07.2023
6,6854981,7090120,,,,,,32,033-00223-2534886,81793,...,0,1,,1,1581116.97,08.11.2024,0,418883.03,,10.10.2024
7,6854981,7090120,,,,,,32,033-00223-2628865,97831,...,0,1,,1,2001112.46,01.11.2024,0,298887.54,,10.10.2024
8,6854981,7090120,,,,,,32,033-00223-2754352,148298,...,,1,12.08.2024,1,0,,0,3400000,,12.08.2024
9,6854981,7090120,,,,,,32,033-00223-2815488,89549,...,,1,,1,2100000,04.11.2024,0,0,,10.10.2024


In [11]:
db = client['task']
task_collection = db['task']
query = {
    'data.katm_333.return.data.general_cbr.loans.loan': {'$exists': True},
    'number': {'$eq': 6682749}
}
projection = {
    'number': 1, 
    'request.clientId': 1,
    'data.katm_333.return.data.general_cbr.loans.loan': 1
}

In [12]:
doc = task_collection.find_one(query, projection)

In [13]:
doc

{'_id': ObjectId('66e85791d0296085df24fa9e'),
 'request': {'clientId': '4916610'},
 'number': 6682749,
 'data': {'katm_333': {'return': {'data': {'general_cbr': {'loans': {'loan': [{'ttl_delq_90_119': '',
         'ttl_delq_8_29': '3',
         'other_past_due': '',
         'collaterals': '',
         'ttl_delq_60_89': '',
         'type': '32',
         'uuid': '033-00837-1907132',
         'initial_pmt': '245378',
         'collateral_code': '51',
         'monthly_details': {'monthly_detail': [{'date': '31.05.2022',
            'next_pmt': '247014.98',
            'pmt_pat': '0',
            'max_delq_balance': '',
            'worst_status': '',
            'outstanding': '4585000',
            'delq_balance': '',
            'history_date': '31.05.2022',
            'status': '',
            'balance_amt': ''},
           {'date': '30.06.2022',
            'next_pmt': '247014.98',
            'pmt_pat': '00',
            'max_delq_balance': '175162.38',
            'worst_status'

In [14]:
rows = []

number = doc.get('number')
client_id = doc.get('request', {}).get('clientId', {})
loans = doc.get('data', {}).get('katm_333', {}).get('return', {}).get('data', {}).get('general_cbr', {}).get('loans', {}).get('loan', [])       

if isinstance(loans, dict):
    loans = [loans]

for loan in loans:
    row = {'client_id': client_id, 'number': number, **loan}
    rows.append(row)

In [1]:
rows

NameError: name 'rows' is not defined

In [15]:
df = pd.DataFrame(rows)

In [16]:
df

,client_id,number,ttl_delq_90_119,ttl_delq_8_29,other_past_due,collaterals,ttl_delq_60_89,type,uuid,initial_pmt,...,principal_past_due,relationship,fact_close_date,is_own,outstanding,next_pmt_principal_date,coborrowers_count,principal_repaid_amount,guarantors,inf_confirm_date
0,4916610,6682749,,3,,,,32,033-00837-1907132,245378,...,0,1,16.05.2024,1,0,,0,4585000,,16.05.2024
1,4916610,6682749,3,2,0,,1,32,033-00478-1533635,198843,...,0,1,16.09.2021,1,0,,0,2135000,,16.09.2021
2,4916610,6682749,,,0,,,32,033-00478-1458477,20571,...,0,1,14.09.2020,1,0,,0,220000,,14.09.2020
3,4916610,6682749,,3,0,,4,32,033-00478-1504193,316512,...,0,1,15.09.2021,1,0,,0,3406000,,15.09.2021
4,4916610,6682749,,,0,,,32,033-00478-1485415,93704,...,0,1,22.10.2020,1,0,,0,1000000,,22.10.2020
5,4916610,6682749,,1,0,,,32,033-00478-1452576,562526,...,0,1,12.08.2021,1,0,,0,6000000,,12.08.2021
6,4916610,6682749,,3,,,,32,033-00837-1939930,98310,...,0,1,22.05.2024,1,0,,0,1857000,,22.05.2024
7,4916610,6682749,,8,,,,30,005-00483-10223672,220691,...,0,1,10.03.2020,0,0,,0,12000000,"{'guarantor': {'open_date': '26.04.18', 'end_d...",10.03.2020
8,4916610,6682749,,10,,"{'collateral': {'open_date': '03.06.19', 'end_...",,32,051-01072-11988,633991,...,0,1,13.07.2020,0,0,,0,12000000,,13.07.2020
9,4916610,6682749,,,,,,32,003-00863-562655,850539,...,,1,,0,,,0,,"{'guarantor': [{'open_date': '26.03.20', 'end_...",


# NEXT TEST

In [43]:
import pandas as pd
from connections import get_mongo_client
from utils import load_columns, map_dff_to_columns, insert_into_mssql

In [44]:
client = client = get_mongo_client()

In [126]:
db = client['task']
task_collection = db['task']
query = {
    'data.katm_077': {'$exists': True},
    'number': {'$gt': 6000000, '$lt': 6000100}
}

projection = {
    '_id': 1,
    'number': 1,
    'data.katm_077.return.data.scorring.scoring_grade': 1,
    'data.katm_077.return.data.scorring.scoring_class': 1,
    'data.katm_077.return.data.scorring.scoring_level': 1,
    'data.katm_077.return.data.scorring.scoring_version': 1
}

In [153]:
docs = task_collection.find(query, projection)

In [128]:
docs

In [150]:
for doc in docs:
    print(doc)


{'_id': ObjectId('66a94d33bd3a41029ebb7ee8'), 'number': 6000002, 'data': {'katm_077': {'return': {'data': {'scorring': {'scoring_version': '1.6', 'scoring_grade': '259', 'scoring_class': 'C2', 'scoring_level': 'СРЕДНИЙ'}}}}}}
{'_id': ObjectId('66a94d42bd3a41029ebb7efd'), 'number': 6000003, 'data': {'katm_077': {'return': {'data': {'scorring': {'scoring_version': '1.6', 'scoring_grade': '126', 'scoring_class': 'D1', 'scoring_level': 'СЛАБЫЙ'}}}}}}
{'_id': ObjectId('66a94d64a5adb329ec84d316'), 'number': 6000004, 'data': {'katm_077': {'return': {'data': {'scorring': {'scoring_version': '1.6', 'scoring_grade': '275', 'scoring_class': 'C3', 'scoring_level': 'СРЕДНИЙ'}}}}}}
{'_id': ObjectId('66a94d78a5adb329ec84d327'), 'number': 6000006, 'data': {'katm_077': {'return': {'data': {'scorring': {'scoring_version': '1.6', 'scoring_grade': '262', 'scoring_class': 'C2', 'scoring_level': 'СРЕДНИЙ'}}}}}}
{'_id': ObjectId('66a94d89bd3a41029ebb7f1a'), 'number': 6000007, 'data': {'katm_077': {'return': 

In [154]:
docs = list(docs)  # Convert the cursor to a list to inspect its content
print(f"Number of documents fetched: {len(docs)}")

Number of documents fetched: 50


In [152]:
docs

[]

In [157]:
rows = []

for doc in docs:
    # Extract fields from the document
    _id = str(doc.get('_id'))  # Convert ObjectId to string
    number = doc.get('number')
    scoring_vals = doc.get('data', {}).get('katm_077', {}).get('return', {}).get('data', {}).get('scorring', {})

    # Merge the extracted fields dynamically
    row = {
        '_id': _id,
        'number': number,
        **scoring_vals  # Dynamically unpack all key-value pairs in scoring_vals
    }
    rows.append(row)
df = pd.DataFrame(rows)

In [158]:
df

,_id,number,scoring_version,scoring_grade,scoring_class,scoring_level
0,66a94d33bd3a41029ebb7ee8,6000002,1.6,259,C2,СРЕДНИЙ
1,66a94d42bd3a41029ebb7efd,6000003,1.6,126,D1,СЛАБЫЙ
2,66a94d64a5adb329ec84d316,6000004,1.6,275,C3,СРЕДНИЙ
3,66a94d78a5adb329ec84d327,6000006,1.6,262,C2,СРЕДНИЙ
4,66a94d89bd3a41029ebb7f1a,6000007,1.6,383,B3,ХОРОШИЙ
5,66a94d8ea5adb329ec84d33b,6000008,1.6,353,B2,ХОРОШИЙ
6,66a94d9fa5adb329ec84d34b,6000010,1.6,325,B1,ХОРОШИЙ
7,66a94df0bd3a41029ebb7f35,6000012,1.6,221,C1,СРЕДНИЙ
8,66a94e3ebd3a41029ebb7f52,6000017,1.6,269,C3,СРЕДНИЙ
9,66a94e80a5adb329ec84d39b,6000020,1.6,260,C2,СРЕДНИЙ


In [156]:
rows

[{'_id': '66a94d33bd3a41029ebb7ee8',
  'number': 6000002,
  'scoring_version': '1.6',
  'scoring_grade': '259',
  'scoring_class': 'C2',
  'scoring_level': 'СРЕДНИЙ'},
 {'_id': '66a94d42bd3a41029ebb7efd',
  'number': 6000003,
  'scoring_version': '1.6',
  'scoring_grade': '126',
  'scoring_class': 'D1',
  'scoring_level': 'СЛАБЫЙ'},
 {'_id': '66a94d64a5adb329ec84d316',
  'number': 6000004,
  'scoring_version': '1.6',
  'scoring_grade': '275',
  'scoring_class': 'C3',
  'scoring_level': 'СРЕДНИЙ'},
 {'_id': '66a94d78a5adb329ec84d327',
  'number': 6000006,
  'scoring_version': '1.6',
  'scoring_grade': '262',
  'scoring_class': 'C2',
  'scoring_level': 'СРЕДНИЙ'},
 {'_id': '66a94d89bd3a41029ebb7f1a',
  'number': 6000007,
  'scoring_version': '1.6',
  'scoring_grade': '383',
  'scoring_class': 'B3',
  'scoring_level': 'ХОРОШИЙ'},
 {'_id': '66a94d8ea5adb329ec84d33b',
  'number': 6000008,
  'scoring_version': '1.6',
  'scoring_grade': '353',
  'scoring_class': 'B2',
  'scoring_level': 'ХОРО

In [145]:
row

{'_id': ('66a94d8ea5adb329ec84d33b',),
 'number': (6000008,),
 'scoring_version': '1.6',
 'scoring_grade': '220',
 'scoring_class': 'C1',
 'scoring_level': 'СРЕДНИЙ'}

In [136]:
rows = []
for doc in docs:

    _id = str(doc.get('_id')) 
    number = doc.get('number')
    scoring_vals = doc.get('data', {}).get('katm_077', {}).get('return', {}).get('data', {}).get('scorring', {})
   
    row = {
        '_id': _id,
        'number': number,
        'scoring_version': scoring_vals.get('scoring_version'),
        'scoring_grade': scoring_vals.get('scoring_grade'),
        'scoring_class': scoring_vals.get('scoring_class'),
        'scoring_level': scoring_vals.get('scoring_level'),
    }
    rows.append(row)
        
df = pd.DataFrame(rows)

In [139]:
row

{'_id': ('66a94d8ea5adb329ec84d33b',),
 'number': (6000008,),
 'scoring_version': '1.6',
 'scoring_grade': '220',
 'scoring_class': 'C1',
 'scoring_level': 'СРЕДНИЙ'}

In [137]:
df

""


In [130]:
rows = []
for doc in docs:
    scoring_vals = doc.get('data', {}).get('katm_077', {}).get('return', {}).get('data', {}).get('scorring', {})
    
    row = {
            '_id': _id,
            'number': number,
            **scoring_vals  # Dynamically unpack all key-value pairs in scoring_vals
        }
    rows.append(row)


In [131]:
df = pd.DataFrame(rows)

In [132]:
df

,_id,number,scoring_version,scoring_grade,scoring_class,scoring_level
0,"(66a94d8ea5adb329ec84d33b,)","(6000008,)",NaN,NaN,NaN,NaN
1,"(66a94d8ea5adb329ec84d33b,)","(6000008,)",NaN,NaN,NaN,NaN
2,"(66a94d8ea5adb329ec84d33b,)","(6000008,)",NaN,NaN,NaN,NaN
3,"(66a94d8ea5adb329ec84d33b,)","(6000008,)",NaN,NaN,NaN,NaN
4,"(66a94d8ea5adb329ec84d33b,)","(6000008,)",1.6,353,B2,ХОРОШИЙ
5,"(66a94d8ea5adb329ec84d33b,)","(6000008,)",1.6,220,C1,СРЕДНИЙ


In [123]:
rows

[{'_id': ('66a94d8ea5adb329ec84d33b',), 'number': (6000008,)},
 {'_id': ('66a94d8ea5adb329ec84d33b',), 'number': (6000008,)},
 {'_id': ('66a94d8ea5adb329ec84d33b',), 'number': (6000008,)},
 {'_id': ('66a94d8ea5adb329ec84d33b',), 'number': (6000008,)},
 {'_id': ('66a94d8ea5adb329ec84d33b',),
  'number': (6000008,),
  'scoring_version': '1.6',
  'scoring_grade': '353',
  'scoring_class': 'B2',
  'scoring_level': 'ХОРОШИЙ'}]

In [55]:
_id

('66a94d33bd3a41029ebb7ee8',)

In [121]:
scoring_vals

{'scoring_version': '1.6',
 'scoring_grade': '353',
 'scoring_class': 'B2',
 'scoring_level': 'ХОРОШИЙ'}

In [63]:
for val in scoring_vals:
    print(**val)

TypeError: print() argument after ** must be a mapping, not str

In [64]:
for val in scoring_vals:
    row = {'_id': _id, 'number': number}
    print(row)

{'_id': ('66a94d33bd3a41029ebb7ee8',), 'number': (6000002,)}
{'_id': ('66a94d33bd3a41029ebb7ee8',), 'number': (6000002,)}
{'_id': ('66a94d33bd3a41029ebb7ee8',), 'number': (6000002,)}
{'_id': ('66a94d33bd3a41029ebb7ee8',), 'number': (6000002,)}


In [54]:

for val in scoring_vals:
    row = {'_id': _id, 'number': number, **val}
    rows.append(row)
    
df = pd.DataFrame(rows)

TypeError: 'str' object is not a mapping

In [27]:
docs = task_collection.find(query, projection)

In [7]:
limited_docs = docs[:20]

In [16]:
for doc in docs:
    data = []
    parsed_doc = {
        '_id': str(doc.get('_id')), 
        'number': doc.get('number'),
        'scoring_grade': doc.get('data', {}).get('katm_077', {}).get('return', {}).get('data', {}).get('scorring', {}).get('scoring_grade'),
        'scoring_class': doc.get('data', {}).get('katm_077', {}).get('return', {}).get('data', {}).get('scorring', {}).get('scoring_class'),
        'scoring_level': doc.get('data', {}).get('katm_077', {}).get('return', {}).get('data', {}).get('scorring', {}).get('scoring_level'),
        'scoring_version': doc.get('data', {}).get('katm_077', {}).get('return', {}).get('data', {}).get('scorring', {}).get('scoring_version')
    }
    data.append(parsed_doc)
    df = pd.DataFrame(data)
    scoring_columns_file = 'katm_077_scoring_columns.txt'
    scoring_columns = load_columns(scoring_columns_file)
    final_df = map_dff_to_columns(df, scoring_columns)
    insert_into_mssql(final_df, 'bronze.katm_077_scoring')


SystemError: <built-in function connect> returned a result with an exception set

In [9]:
scoring_columns_file = 'katm_077_scoring_columns.txt'
scoring_columns = load_columns(scoring_columns_file)
final_df = map_dff_to_columns(df, scoring_columns)
insert_into_mssql(final_df, 'bronze.katm_077_scoring')

In [120]:
for doc in docs:
    print(doc)

In [ ]:
for doc in docs:
    rows = []
    _id = str(doc.get('_id')), 
    number = doc.get('number'),
    scoring_vals = doc.get('data', {}).get('katm_077', {}).get('return', {}).get('data', {}).get('scorring', {}).get('scoring_grade', [])
   
    for val in scoring_vals:
        row = {'_id': _id, 'number': number, **val}
        rows.append(row)
        
    df = pd.DataFrame(rows)
    
    scoring_columns_file = 'katm_077_scoring_columns.txt'
    scoring_columns = load_columns(scoring_columns_file)
    final_df = map_dff_to_columns(df, scoring_columns)
    insert_into_mssql(final_df, 'bronze.katm_077_scoring')


In [30]:
from connections import get_sql_server_connection

In [40]:
def number_is_present(table_name, num):
    conn = get_sql_server_connection()
    cursor = conn.cursor()

    # Check if table exists, if not create it
    check_table_query = f"""select 1 from {table_name} where number = {num};
    """
    cursor.execute(check_table_query)
    result = cursor.fetchone()
    my_num = int(result[0] if result else None)
    return my_num

In [42]:
my_val = number_is_present('bronze.katm_077_scoring', '8092370')
print(my_val)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [ ]:
for doc in docs:
    rows = []
    _id = str(doc.get('_id')), 
    number = doc.get('number'),
    scoring_vals = doc.get('data', {}).get('katm_077', {}).get('return', {}).get('data', {}).get('scorring', {}).get('scoring_grade', [])
   
    for val in scoring_vals:
        row = {'_id': _id, 'number': number, **val}
        rows.append(row)
        
    df = pd.DataFrame(rows)
    
    scoring_columns_file = 'katm_077_scoring_columns.txt'
    scoring_columns = load_columns(scoring_columns_file)
    final_df = map_dff_to_columns(df, scoring_columns)
    
    insert_into_mssql(final_df, 'bronze.katm_077_scoring')